# Lab 11: Ransomware Detection & Response

Build an AI-powered system to detect, analyze, and respond to ransomware attacks.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab11_ransomware_detection.ipynb)

## Learning Objectives
- Ransomware behavioral detection
- Entropy-based encryption detection
- Ransom note analysis
- Automated response playbooks

In [ ]:
import math
import re
from collections import Counter
from dataclasses import dataclass
from typing import List, Dict

## 1. Entropy-Based Detection

In [ ]:
def calculate_entropy(data: bytes) -> float:
    """Calculate Shannon entropy of data."""
    if not data:
        return 0.0
    
    counter = Counter(data)
    length = len(data)
    entropy = 0.0
    
    for count in counter.values():
        probability = count / length
        if probability > 0:
            entropy -= probability * math.log2(probability)
    
    return entropy

# Test entropy
test_cases = [
    ("Normal text", b"Hello, this is a normal text file."),
    ("Repeated bytes", b"AAAAAAAAAAAAAAAA"),
    ("Random (encrypted)", bytes(range(256)) * 4)
]

print("Entropy Analysis:")
print("=" * 40)
for name, data in test_cases:
    entropy = calculate_entropy(data)
    status = "HIGH (encrypted?)" if entropy > 7.0 else "NORMAL"
    print(f"{name}: {entropy:.2f} - {status}")

## 2. File Event Analysis

In [ ]:
@dataclass
class FileEvent:
    """File system event."""
    timestamp: str
    process: str
    operation: str
    file_path: str
    extension: str
    entropy: float

# Sample ransomware events
SAMPLE_EVENTS = [
    FileEvent("09:00:00", "explorer.exe", "READ", "C:\\doc.docx", ".docx", 4.5),
    FileEvent("09:00:05", "malware.exe", "WRITE", "C:\\doc.docx.locked", ".locked", 7.98),
    FileEvent("09:00:06", "malware.exe", "WRITE", "C:\\data.xlsx.locked", ".locked", 7.95),
    FileEvent("09:00:07", "cmd.exe", "EXECUTE", "vssadmin delete shadows", "", 0),
    FileEvent("09:00:08", "malware.exe", "CREATE", "C:\\README_DECRYPT.txt", ".txt", 3.5),
]

# Ransomware extensions
RANSOMWARE_EXTENSIONS = {
    '.locked', '.encrypted', '.crypto', '.crypt',
    '.locky', '.lockbit', '.alphv', '.phobos'
}

class RansomwareDetector:
    """Detect ransomware behavior."""
    
    ENTROPY_THRESHOLD = 7.5
    
    def analyze(self, events: List[FileEvent]) -> Dict:
        result = {
            "is_ransomware": False,
            "confidence": 0.0,
            "indicators": [],
            "mitre_techniques": []
        }
        
        score = 0.0
        
        # Check for high entropy writes
        high_entropy = [e for e in events if e.operation == "WRITE" and e.entropy > self.ENTROPY_THRESHOLD]
        if high_entropy:
            score += 0.4
            result["indicators"].append(f"{len(high_entropy)} high-entropy writes")
            result["mitre_techniques"].append("T1486 - Data Encrypted for Impact")
        
        # Check for ransomware extensions
        ransom_ext = [e for e in events if e.extension.lower() in RANSOMWARE_EXTENSIONS]
        if ransom_ext:
            score += 0.3
            result["indicators"].append(f"{len(ransom_ext)} ransomware extensions")
        
        # Check for shadow copy deletion
        shadow_del = [e for e in events if "vssadmin" in e.file_path.lower()]
        if shadow_del:
            score += 0.3
            result["indicators"].append("Shadow copy deletion")
            result["mitre_techniques"].append("T1490 - Inhibit System Recovery")
        
        result["confidence"] = min(score, 1.0)
        result["is_ransomware"] = score >= 0.7
        
        return result

# Detect
detector = RansomwareDetector()
result = detector.analyze(SAMPLE_EVENTS)

print("Ransomware Detection Results:")
print(f"  Detected: {result['is_ransomware']}")
print(f"  Confidence: {result['confidence']:.0%}")
print(f"  Indicators: {result['indicators']}")
print(f"  MITRE: {result['mitre_techniques']}")

## 3. Ransom Note Analysis

In [ ]:
SAMPLE_NOTE = """
YOUR FILES HAVE BEEN ENCRYPTED BY LOCKBIT 3.0

All important files are encrypted. To recover your files:

1. Send 0.5 BTC to: bc1qxy2kgdygjrsqtzq2n0yrf2493p83kkfjhx0wlh
2. Contact us: support@lockbit.onion
3. Visit: lockbitapt6vx57t3eeqjofwgcglmutr3a35nygvokja5uuccip4ykyd.onion

You have 72 hours before the price doubles.
DO NOT contact law enforcement or recovery companies.
"""

class RansomNoteAnalyzer:
    """Extract IOCs from ransom notes."""
    
    BTC_PATTERN = r'\b(bc1|[13])[a-zA-HJ-NP-Z0-9]{25,39}\b'
    ONION_PATTERN = r'\b[a-z2-7]{16,56}\.onion\b'
    EMAIL_PATTERN = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    
    def extract_iocs(self, note: str) -> Dict:
        return {
            "bitcoin": re.findall(self.BTC_PATTERN, note),
            "onion": re.findall(self.ONION_PATTERN, note, re.IGNORECASE),
            "email": re.findall(self.EMAIL_PATTERN, note)
        }
    
    def identify_family(self, note: str) -> str:
        """Identify ransomware family."""
        families = {
            "lockbit": ["lockbit", "lockbit 3.0"],
            "blackcat": ["blackcat", "alphv"],
            "conti": ["conti"],
            "revil": ["revil", "sodinokibi"]
        }
        
        note_lower = note.lower()
        for family, keywords in families.items():
            if any(kw in note_lower for kw in keywords):
                return family
        return "unknown"

# Analyze note
analyzer = RansomNoteAnalyzer()
iocs = analyzer.extract_iocs(SAMPLE_NOTE)
family = analyzer.identify_family(SAMPLE_NOTE)

print("Ransom Note Analysis:")
print(f"  Family: {family.upper()}")
print(f"  Bitcoin addresses: {len(iocs['bitcoin'])}")
print(f"  Onion URLs: {len(iocs['onion'])}")
print(f"  Emails: {iocs['email']}")

## 4. Response Playbook

In [ ]:
def generate_playbook(detection_result: Dict, iocs: Dict) -> List[Dict]:
    """Generate ransomware response playbook."""
    playbook = [
        {
            "priority": 1,
            "action": "ALERT",
            "description": f"Ransomware detected (confidence: {detection_result['confidence']:.0%})",
            "automated": True
        },
        {
            "priority": 2,
            "action": "ISOLATE_HOST",
            "description": "Network isolate affected hosts immediately",
            "automated": False
        },
        {
            "priority": 3,
            "action": "PRESERVE_EVIDENCE",
            "description": "Capture memory dump and disk image",
            "automated": False
        }
    ]
    
    # Add IOC blocking
    if iocs.get("bitcoin"):
        playbook.append({
            "priority": 4,
            "action": "BLOCK_IOCS",
            "description": f"Block {len(iocs['bitcoin'])} bitcoin addresses in monitoring",
            "automated": True
        })
    
    if iocs.get("onion"):
        playbook.append({
            "priority": 5,
            "action": "BLOCK_TOR",
            "description": "Block TOR exit nodes if not already",
            "automated": True
        })
    
    playbook.append({
        "priority": 10,
        "action": "RECOVERY_ASSESSMENT",
        "description": "Assess backup integrity and recovery options",
        "automated": False
    })
    
    return playbook

# Generate playbook
playbook = generate_playbook(result, iocs)

print("\nResponse Playbook:")
print("=" * 50)
for step in playbook:
    auto = "[AUTO]" if step["automated"] else "[MANUAL]"
    print(f"{step['priority']}. {auto} {step['action']}")
    print(f"   {step['description']}")

## Summary

We built a ransomware detection system with:

1. **Entropy Analysis** - Detect encrypted files
2. **Behavior Detection** - Identify ransomware patterns
3. **Ransom Note Analysis** - Extract IOCs
4. **Response Playbook** - Automated actions

### Key Indicators:
- High entropy writes (> 7.5)
- Known ransomware extensions
- Shadow copy deletion
- Ransom note creation

### Next Steps:
1. Add real-time file monitoring
2. Implement ML-based family classification
3. Create automated backup verification